In [1]:
#Package Manager Install
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [20]:
#THIS SHEET WILL BE USED TO DEFINE SUPPORT FUNCTIONS BEFORE WE BEGIN TO MAKE API CALLS AND TRANSFORM THE DATA
#Library Imports
import numpy as np
import yfinance as yf
import random
import pandas as pd

# GLOBAL FUNCTIONS
#Takes daily sigma and returns a yearly sigma value

x = 4
def sigma_yearly (sigma):
    return sigma * (252 ** 0.5)

# Takes inputs to Sharpe Equation and returns ratio

def calc_sharpe(ERP, riskFreeRate, sigmaPortfolioYearly):
    sharpe = (ERP - riskFreeRate) / sigmaPortfolioYearly
    return sharpe

# Will be used to multiply Covariance Matrix by Weighting Matrix to return variance, can also be used to find the expected return of the portfolio (w * ER_i)

def sum_product (list1,list2):
    var_list = []
    for i in range(0, len(list1)):
        var_list.append(list1[i] * list2[i])
    return sum(var_list)

# Preforms a linear regression

def calc_beta(xlist, ylist):
    # Perform linear regression
    beta, _ = np.polyfit(xlist, ylist, 1)
    return beta

def ERI_CAPM (r_f,ERP,beta):
    return r_f + ERP * beta


# Oldest data will be returned first [price_day1, price_day2, etc..]
def daily_adj_close_list (ticker_string):
    ticker = yf.Ticker(ticker_string)
    historical_data = ticker.history(period='1y')
    return historical_data['Close'].tolist()

def calc_daily_returns(stock_price_list):
    daily_returns = [(stock_price_list[i] / stock_price_list[i-1]) - 1 for i in range(1, len(stock_price_list))]
    return daily_returns

def current_treasury_rate():
    treasury_rate = yf.Ticker('^TNX')
    return treasury_rate.history(period='5d')['Close'].iloc[0]/100

# This uses sample covariance instead of population covariance as recommended by Prof.
def calculate_covariance_matrix(list_of_lists):
    num_lists = len(list_of_lists)
    covariances = []
    
    for i in range(num_lists):
        for j in range(num_lists):
            list1 = list_of_lists[i]
            list2 = list_of_lists[j]
            
            if len(list1) != len(list2):
                raise ValueError("All lists must have the same length")
            
            mean_list1 = np.mean(list1)
            mean_list2 = np.mean(list2)
            
            deviations_list1 = list1 - mean_list1
            deviations_list2 = list2 - mean_list2
            
            covariance = np.sum(deviations_list1 * deviations_list2) / (len(list1) - 1)
            covariances.append(covariance)
    
    return covariances

def create_weighting_list(beta_list):
    global weighting_list
    weighting_list = []
    total_weight = 0
    for i in range(len(beta_list)):
        if i == len(beta_list) - 1:
            weight = 1 - total_weight  # Ensure the last weight makes the sum equal to 1
        else:
            weight = random.uniform(0, 1 - total_weight)
        total_weight += weight
        globals()['weight_{}'.format(i+1)] = weight
        weighting_list.append(globals()['weight_{}'.format(i+1)])

def create_matrix(input_list):
    output_list = []
    for i in input_list:
        for j in input_list:
            output_list.append(i * j)
    return output_list 
        
# This will be where the main code of the program runs
# Step 1 - ask users for ticker inputs

# Creates ticker list after asking the user for tickers
ticker_input = input("Enter stock tickers separated by a comma: ")
ticker_list = ticker_input.split(',')
ticker_list = [ticker.strip() for ticker in ticker_list]

# Step 2 - take all symbols and create a list of lists with daily returns
daily_return_list = []
for symbol in ticker_list:
    list = calc_daily_returns(daily_adj_close_list(symbol))
    daily_return_list.append(list)
    
# Step 3 - create a list for the daily return series of the market (S&P 500 Total Return)
benchmark_return_list = calc_daily_returns(daily_adj_close_list('^SPX'))

# Step 4 - beta calculation comparing all items lists in the daily return list with the benchmark return list
beta_list = []
for return_series in daily_return_list:
    beta = calc_beta(benchmark_return_list,return_series)
    beta_list.append(beta)

# Step 5 - creating a CAPM list
capm_list = []
r_f = current_treasury_rate()
ERP = 0.0457
for beta in beta_list:
    ER_i = r_f + beta * ERP
    capm_list.append(ER_i)

# Step 6 - Creating the covariance matrix
covariance_matrix = calculate_covariance_matrix(daily_return_list)

# Step 7 - Creating the weighting matrix - the weighting_list is a global variable that will be interated over in a solver like fashion in a later function
create_weighting_list(beta_list)


# Step 8 - Create the cycle code that needs to be recalculated every time that the global weighting_list is changed
def calculate_sharpe(weighting_list, covariance_matrix, capm_list, r_f):
    weighting_matrix = create_matrix(weighting_list)
    var_p = sum_product(weighting_matrix, covariance_matrix)
    sigma_p = var_p ** 0.5
    sigma_p_yearly = sigma_yearly(sigma_p)
    ER_p = sum_product(weighting_list, capm_list)
    
    if sigma_p_yearly != 0:
        sharpe = (ER_p - r_f) / sigma_p_yearly
    else:
        sharpe = np.nan  # Set Sharpe ratio to NaN if sigma_p_yearly is zero
    return sharpe

def maximize_sharpe(weighting_list, covariance_matrix, capm_list, r_f, iterations=100000):
    max_sharpe = float('-inf')
    max_weights = None
    
    for _ in range(iterations):
        new_weights = np.random.dirichlet(np.ones(len(weighting_list)))
        current_sharpe = calculate_sharpe(new_weights, covariance_matrix, capm_list, r_f)
        if current_sharpe > max_sharpe:
            max_sharpe = current_sharpe
            max_weights = new_weights
    
    return max_sharpe, max_weights

# Step 9 - Display
max_sharpe, max_weights = maximize_sharpe(weighting_list, covariance_matrix, capm_list, r_f, iterations=100000)

rounded_weights = [round(w, 4) for w in max_weights]

# Create a DataFrame to display the results
results_df = pd.DataFrame({'Ticker Symbol': ticker_list, 'Weighting': rounded_weights}, index=range(1, len(ticker_list) + 1))

# Display the DataFrame without listing numbers on the left
display(results_df)

# Print the Sharpe ratio below the table in a professional format
print(f"Max Sharpe Ratio: {max_sharpe:.4f}")

# weighting_list = [0.513,0.0465,0.0405,0.25,0.15]
# fid = calculate_sharpe(weighting_list, covariance_matrix, capm_list, r_f)
# print(fid)

Enter stock tickers separated by a comma:  AAPL, CMG, TSLA, DIS, AMZN, GOOGL, MSFT, NFLX, INTC, AMD, NVDA, ADBE, CRM, ORCL, CSCO, IBM, HPQ, DELL, ACN, CTSH, IBM, ADP, INTU, PAYX, DXC, WDC, STX, NTAP, XRX, TEL, T, CCI, VZ, TMUS, AMT, SBAC, MSI, ERIC, BCE, TU, RCI, TD, NVDA, AMD, EA, TTWO, UBER, LYFT, EBAY, ETSY


,Ticker Symbol,Weighting
1,AAPL,0.0590
2,CMG,0.0535
3,TSLA,0.0285
4,DIS,0.0088
5,AMZN,0.0145
6,GOOGL,0.0210
7,MSFT,0.0048
8,NFLX,0.0038
9,INTC,0.0152
10,AMD,0.0028


Max Sharpe Ratio: 0.3528
